<a href="https://colab.research.google.com/github/marcosferreiracabral/projeto_pyspark_aviacao/blob/main/avaliacao_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Instalar as dependências:

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark


In [3]:
# Configuração das variáveis de ambiente:

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

# Tornar o pyspark "importável":

import findspark

findspark.init('spark-2.4.4-bin-hadoop2.7')


In [4]:
# Iniciar uma sessão local:

import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()


In [5]:
# Importação das bibliotecas para o notebook:

from pyspark.sql.functions import col, StringType, lit, expr, substring
from pyspark.sql import Window
from pyspark.sql import functions as func


# Airports Dataset:

In [6]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/1cpygs-tfjUqAYqbBAkt3zcVP1h4J_3W_/view?usp=sharing

# Carregar dados do Airports:

df_airports = spark.read.csv("./airports.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_airports.printSchema()


view?usp=sharing.19     [ <=>                ]  67.50K  --.-KB/s    in 0.003s  
root
 |-- faa: string (nullable = true)
 |-- name: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- alt: integer (nullable = true)
 |-- tz: integer (nullable = true)
 |-- dst: string (nullable = true)



In [7]:
# Lendo o dataframe e impimindo as colunas e linhas:

df_airports.show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

Perguntas<br>
Considere o dataset airports.csv para realizar as seguintes tarefas:

In [8]:
df_airports.head(10)


[Row(faa='04G', name='Lansdowne Airport', lat=41.1304722, lon=-80.6195833, alt=1044, tz=-5, dst='A'),
 Row(faa='06A', name='Moton Field Municipal Airport', lat=32.4605722, lon=-85.6800278, alt=264, tz=-5, dst='A'),
 Row(faa='06C', name='Schaumburg Regional', lat=41.9893408, lon=-88.1012428, alt=801, tz=-6, dst='A'),
 Row(faa='06N', name='Randall Airport', lat=41.431912, lon=-74.3915611, alt=523, tz=-5, dst='A'),
 Row(faa='09J', name='Jekyll Island Airport', lat=31.0744722, lon=-81.4277778, alt=11, tz=-4, dst='A'),
 Row(faa='0A9', name='Elizabethton Municipal Airport', lat=36.3712222, lon=-82.1734167, alt=1593, tz=-4, dst='A'),
 Row(faa='0G6', name='Williams County Airport', lat=41.4673056, lon=-84.5067778, alt=730, tz=-5, dst='A'),
 Row(faa='0G7', name='Finger Lakes Regional Airport', lat=42.8835647, lon=-76.7812318, alt=492, tz=-5, dst='A'),
 Row(faa='0P2', name='Shoestring Aviation Airfield', lat=39.7948244, lon=-76.6471914, alt=1000, tz=-5, dst='U'),
 Row(faa='0S9', name='Jefferson 

In [9]:
df_airports.select(col('faa')).count()


1397

In [10]:
df_airports.select(col('faa')).dtypes


[('faa', 'string')]

In [11]:
df_airports.select(col('faa')).collect()


[Row(faa='04G'),
 Row(faa='06A'),
 Row(faa='06C'),
 Row(faa='06N'),
 Row(faa='09J'),
 Row(faa='0A9'),
 Row(faa='0G6'),
 Row(faa='0G7'),
 Row(faa='0P2'),
 Row(faa='0S9'),
 Row(faa='0W3'),
 Row(faa='10C'),
 Row(faa='17G'),
 Row(faa='19A'),
 Row(faa='1A3'),
 Row(faa='1B9'),
 Row(faa='1C9'),
 Row(faa='1CS'),
 Row(faa='1G3'),
 Row(faa='1OH'),
 Row(faa='1RL'),
 Row(faa='24C'),
 Row(faa='24J'),
 Row(faa='25D'),
 Row(faa='29D'),
 Row(faa='2A0'),
 Row(faa='2G2'),
 Row(faa='2G9'),
 Row(faa='2J9'),
 Row(faa='369'),
 Row(faa='36U'),
 Row(faa='38W'),
 Row(faa='3D2'),
 Row(faa='3G3'),
 Row(faa='3G4'),
 Row(faa='3J1'),
 Row(faa='3W2'),
 Row(faa='40J'),
 Row(faa='41N'),
 Row(faa='47A'),
 Row(faa='49A'),
 Row(faa='49X'),
 Row(faa='4A4'),
 Row(faa='4A7'),
 Row(faa='4A9'),
 Row(faa='4B8'),
 Row(faa='4G0'),
 Row(faa='4G2'),
 Row(faa='4G4'),
 Row(faa='4I7'),
 Row(faa='4U9'),
 Row(faa='52A'),
 Row(faa='54J'),
 Row(faa='55J'),
 Row(faa='57C'),
 Row(faa='60J'),
 Row(faa='6A2'),
 Row(faa='6K8'),
 Row(faa='6S0'

In [12]:
df_airports.select(col('faa')).distinct().count()


1396

In [13]:
df_airports.select(col('faa')).explain()


== Physical Plan ==
*(1) FileScan csv [faa#10] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/content/airports.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<faa:string>


In [14]:
df_airports.fillna('').show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [15]:
df_airports.fillna(value='', subset=['faa']).show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [16]:
df_airports.dropna().show()


+---+--------------------+----------------+-----------------+----+---+---+
|faa|                name|             lat|              lon| alt| tz|dst|
+---+--------------------+----------------+-----------------+----+---+---+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|
|0S9|Jefferson County ...|      48.0538086|     -122.8106436| 108| -8|  A|
|0W3|Harford County Ai...

In [17]:
df_airports.describe(['faa']).show()


+-------+------------------+
|summary|               faa|
+-------+------------------+
|  count|              1397|
|   mean|             141.0|
| stddev|197.46392075515973|
|    min|               04G|
|    max|               ZYP|
+-------+------------------+



1 - Crie a coluna qa_faa e aponte inconsistências da coluna faa de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formator de 3-5 caracteres alfanuméricos.

In [18]:
df_airports_qa_faa = df_airports.withColumn('qa_faa', lit(col('faa') < 5).isNull().cast(StringType())).show()


+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_faa|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|  true|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|  true|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|  true|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|  true|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|  true|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|  true|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|  true|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|  true|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|  true|
|0S9|Jefferson C

2 - Crie a coluna qa_name e aponte inconsistências da coluna name de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.

In [19]:
df_airports_qa_name = df_airports.withColumn('qa_name', lit(col('name') != '').isNull().cast(StringType())).show()


+---+--------------------+----------------+-----------------+----+---+---+-------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_name|
+---+--------------------+----------------+-----------------+----+---+---+-------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|  false|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|  false|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|  false|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|  false|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|  false|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|  false|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|  false|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492| -5|  A|  false|
|0P2|Shoestring Aviati...|      39.7948244|      -76.6471914|1000| -5|  U|  false|
|0S9

3 - Crie a coluna qa_lat e aponte inconsistências da coluna lat de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo .<br>
A : Indica que o valor é alfanumérico.

In [20]:
df_airports.filter(col("lat")
    .rlike("^[0-9]*$").isNull()
  ).show()
  
df_airports_qa_lat = df_airports.withColumn('qa_lat', lit(col('lat') <= 11).isNull().cast(StringType())).show()


+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+

+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lat|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A| false|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A| false|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A| false|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A| false|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A| false|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A| false|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A| false|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.78

4 - Crie a coluna qa_lon e aponte inconsistências da coluna lon de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo .<br>
A : Indica que o valor é alfanumérico.

In [21]:
df_airports.filter(col("lon")
    .rlike("^[0-9]*$").isNull()
  ).show()
  
df_airports_qa_lon = df_airports.withColumn('qa_lon', lit(col('lon') <= 16).isNull().cast(StringType())).show()


+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+

+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_lon|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A| false|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A| false|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A| false|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A| false|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A| false|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A| false|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A| false|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.78

5 - Crie a coluna qa_alt e aponte inconsistências da coluna alt de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [0,+∞).<br>
A : Indica que o valor é alfanumérico.<br>

In [22]:
df_airports.filter(col("alt")
    .rlike("^[0-9]*$").isNull()
  ).show()
  
df_airports_qa_alt = df_airports.withColumn('qa_alt', lit(col('alt') <= 0).isNull().cast(StringType())).show()


+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+

+---+--------------------+----------------+-----------------+----+---+---+------+
|faa|                name|             lat|              lon| alt| tz|dst|qa_alt|
+---+--------------------+----------------+-----------------+----+---+---+------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A| false|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A| false|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A| false|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A| false|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A| false|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A| false|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A| false|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.78

6 - Crie a coluna qa_tz e aponte inconsistências da coluna tz de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo .<br>
A : Indica que o valor é alfanumérico.

In [23]:
df_airports.filter(col("tz")
    .rlike("^[0-9]*$").isNull()
  ).show()
  
df_airports_qa_tz = df_airports.withColumn('qa_tz', lit(col('tz') <= 0).isNull().cast(StringType())).show()


+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+

+---+--------------------+----------------+-----------------+----+---+---+-----+
|faa|                name|             lat|              lon| alt| tz|dst|qa_tz|
+---+--------------------+----------------+-----------------+----+---+---+-----+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|false|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|false|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|false|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|false|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|false|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|false|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|false|
|0G7|Finger Lakes Regi...|      42.8835647|      -76.7812318| 492

7 - Crie a coluna qa_dst e aponte inconsistências da coluna dst de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma das categorias esperadas: E, A, S, O, Z, N, U<br>
N : Indica que o valor é numérico.

In [24]:
df_airports.filter(col("dst")
    .rlike("^[0-9]*$").isNull()
  ).show()

df_airports_qa_dst = df_airports.withColumn('qa_dst',
                                            expr("CASE WHEN dst = 'E' THEN 'não pertence' "
                                            + "WHEN dst = 'A' THEN 'não pertence'" + "WHEN dst = 'S' THEN 'não pertence'"
                                            + "WHEN dst = 'O' THEN 'não pertence'" + "WHEN dst = 'Z' THEN 'não pertence'"
                                            + "WHEN dst = 'N' THEN 'não pertence'" + "WHEN dst = 'U' THEN 'não pertence'"
                                            "ELSE dst END")).show()


+---+----+---+---+---+---+---+
|faa|name|lat|lon|alt| tz|dst|
+---+----+---+---+---+---+---+
+---+----+---+---+---+---+---+

+---+--------------------+----------------+-----------------+----+---+---+------------+
|faa|                name|             lat|              lon| alt| tz|dst|      qa_dst|
+---+--------------------+----------------+-----------------+----+---+---+------------+
|04G|   Lansdowne Airport|      41.1304722|      -80.6195833|1044| -5|  A|não pertence|
|06A|Moton Field Munic...|      32.4605722|      -85.6800278| 264| -5|  A|não pertence|
|06C| Schaumburg Regional|      41.9893408|      -88.1012428| 801| -6|  A|não pertence|
|06N|     Randall Airport|       41.431912|      -74.3915611| 523| -5|  A|não pertence|
|09J|Jekyll Island Air...|      31.0744722|      -81.4277778|  11| -4|  A|não pertence|
|0A9|Elizabethton Muni...|      36.3712222|      -82.1734167|1593| -4|  A|não pertence|
|0G6|Williams County A...|      41.4673056|      -84.5067778| 730| -5|  A|não perte

# Planes Dataset:

In [25]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/18Ru1UuLHv9d6nPBWj5-T_ikyJAHhaDI2/view?usp=sharing

# Carregar dados do Planes:

df_planes = spark.read.csv("./planes.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_planes.printSchema()


view?usp=sharing.20     [ <=>                ]  67.48K  --.-KB/s    in 0.004s  
root
 |-- tailnum: string (nullable = true)
 |-- year: string (nullable = true)
 |-- type: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- model: string (nullable = true)
 |-- engines: integer (nullable = true)
 |-- seats: integer (nullable = true)
 |-- speed: string (nullable = true)
 |-- engine: string (nullable = true)



In [26]:
df_planes.show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

In [27]:
df_planes.fillna('').show()


+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
|tailnum|year|                type|    manufacturer|   model|engines|seats|speed|   engine|
+-------+----+--------------------+----------------+--------+-------+-----+-----+---------+
| N102UW|1998|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N103US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N104UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N105UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N107US|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N108UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N109UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA|Turbo-fan|
| N110UW|1999|Fixed wing multi ...|AIRBUS INDUSTRIE|A320-214|      2|  182|   NA

Perguntas<br>
Considere o dataset planes.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_tailnum e aponte inconsistências da coluna tailnum de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
S : Indica que não tem exatamente 5 caracteres.<br>
F : Indica que não respeita o formato esperado (ex. N1234Z ou N123AZ).<br>
FN : Indica que não inicia com a letra "N".<br>
FE : Indica que contém caracteres inválidos ("I", "O", ou 0 como primeiro digito).<br>

In [28]:
df_planes.filter(col("tailnum")
    .rlike("^[0-9]*$").isNull()
  ).show()

df_planes.select(expr("length(tailnum)")).show()

df_planes.select(substring('tailnum', 0, 1).cast(StringType()).alias('qa_tailnum')).show()

expression = """("CASE WHEN tailnum = 'I' THEN 'não pertence' "
                + "WHEN tailnum = 'O' THEN 'não pertence'"
                + "WHEN tailnum = 0 THEN 'não pertence'"
                "ELSE tailnum END")"""

df_planes.filter(col("tailnum")
                .rlike("N")
                ).show()


+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+

+---------------+
|length(tailnum)|
+---------------+
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
|              6|
+---------------+
only showing top 20 rows

+----------+
|qa_tailnum|
+----------+
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|         N|
|   

2 - Crie a coluna qa_year e aponte inconsistências da coluna year de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [1950,+∞).

In [29]:
df_planes.filter(col("year")
                .rlike("^[0-9]*$").isNull()
                ).show()

window = Window.partitionBy("qa_year").orderBy("year").rangeBetween(Window.currentRow, 1950)

# df_planes.withColumn("qa_year", window).show()


+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+



3 - Crie a coluna qa_type e aponte inconsistências da coluna type de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
Fixed wing multi engine<br>
Fixed wing single engine<br>
Rotorcraft

In [30]:
df_planes.filter(col("type")
                .isNull()
                ).show()
  
expression = """("CASE WHEN type = 'Fixed wing multi engine' THEN 'não pertence' "
                + "WHEN type = 'Fixed wing single engine' THEN 'não pertence'"
                + "WHEN type = 'Rotorcraft' THEN 'não pertence'"
                "ELSE type END")"""


+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+



4 - Crie a coluna qa_manufacturer e aponte inconsistências da coluna manufacturer de acordo com
as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
AIRBUS<br>
BOEING<br>
BOMBARDIER<br>
CESSNA<br>
EMBRAER<br>
SIKORSKY<br>
CANADAIR<br>
PIPER<br>
MCDONNELL DOUGLAS<br>
CIRRUS<br>
BELL<br>
KILDALL GARY<br>
LAMBERT RICHARD<br>
BARKER JACK<br>
ROBINSON HELICOPTER<br>
GULFSTREAM<br>
MARZ BARRY<br>

In [31]:
df_planes.filter(col("manufacturer")
                .isNull()
                ).show()


expression = """("CASE WHEN manufacturer = 'AIRBUS' THEN 'pertence'"
                + "WHEN manufacturer = 'BOEING' THEN 'pertence'"
                + "WHEN manufacturer = 'BOMBARDIER' THEN 'pertence'"
                + "WHEN manufacturer = 'CESSNA' THEN 'pertence'"
                + "WHEN manufacturer = 'EMBRAER' THEN 'pertence'"
                + "WHEN manufacturer = 'SIKORSKY' THEN 'pertence'"
                + "WHEN manufacturer = 'CANADAIR' THEN 'pertence'"
                + "WHEN manufacturer = 'PIPER' THEN 'pertence'"
                + "WHEN manufacturer = 'MCDONNELL DOUGLAS' THEN 'pertence'"
                + "WHEN manufacturer = 'CIRRUS' THEN 'pertence'"
                + "WHEN manufacturer = 'BELL' THEN 'pertence'"
                + "WHEN manufacturer = 'KILDALL GARY' THEN 'pertence'"
                + "WHEN manufacturer = 'LAMBERT RICHARD' THEN 'pertence'"
                + "WHEN manufacturer = 'BARKER JACK' THEN 'pertence'"
                + "WHEN manufacturer = 'ROBINSON HELICOPTER' THEN 'pertence'"
                + "WHEN manufacturer = 'GULFSTREAM' THEN 'pertence'"
                + "WHEN manufacturer = 'MARZ BARRY' THEN 'pertence'"
                "ELSE manufacturer END")"""


+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+



5 - Crie a coluna qa_model e aponte inconsistências da coluna model de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado<br>
Modelos AIRBUS devem começar com "A"<br>
Modelos BOEING devem começar com "7"<br>
Modelos BOMBARDIER e CANADAIR devem começar com "CL"<br>
Modelos MCDONNELL DOUGLAS devem começar com "MD" ou "DC"

In [32]:
df_planes.filter(col("model")
                .rlike("^[0-9]*$").isNull()
                ).show()

df_planes.filter(col("model")
                .rlike("A") == ' AIRBUS'
                ).show()

df_planes.filter(col("model")
                .rlike("7") == 'BOEING'
                ).show()

df_planes.filter(col("model")
                .rlike("CL") == 'BOMBARDIER'
                ).show()

df_planes.filter(col("model")
                .rlike("CL") == 'CANADAIR'
                ).show()
 
df_planes.filter(col("model")
                .rlike("MD") == 'MCDONNELL DOUGLAS'
                ).show()

df_planes.filter(col("model")
                .rlike("DC") == 'MCDONNELL DOUGLAS'
                ).show()
 

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+------

6 - Crie a coluna qa_engines e aponte inconsistências da coluna engines de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [1, 4].<br>
A : Indica que o valor é alfanumérico.

In [33]:
df_planes.filter(col("engines")
                .rlike("^[0-9]*$").isNull()
                ).show()

df_planes.select(substring('engines', 1, 4).cast(StringType()).alias('qa_engines')).show()

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+

+----------+
|qa_engines|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
+----------+
only showing top 20 rows



7 - Crie a coluna qa_seats e aponte inconsistências da coluna seats de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br><br>
I : Indica que o valor excede o intervalo .<br>
A : Indica que o valor é alfanumérico.

In [34]:
df_planes.filter(col("seats")
                .rlike("^[0-9]*$").isNull()
                ).show()
                

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+



8 - Crie a coluna qa_speed e aponte inconsistências da coluna speed de acordo com as regras abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [50, 150].<br>
A : Indica que o valor é alfanumérico.<br>

In [35]:
df_planes.filter(col("speed")
                .rlike("^[0-9]*$").isNull()
                ).show()

df_planes.select(substring('speed', 50, 150).cast(StringType()).alias('qa_speed')).show()

+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+

+--------+
|qa_speed|
+--------+
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
|        |
+--------+
only showing top 20 rows



9 - Crie a coluna qa_engine e aponte inconsistências da coluna engine de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
C : Indica que o valor não pertence a nenhuma categoria esperada:<br>
Turbo-fan<br>
Turbo-jet<br>
Turbo-prop<br>
Turbo-shaft<br>
4 Cycle

In [36]:
df_planes.filter(col("engine")
                .isNull()
                ).show()

expression = """("CASE WHEN engine = 'Turbo-fan' THEN 'não pertence' "
                + "WHEN engine = 'Turbo-jet' THEN 'não pertence'"
                + "WHEN engine = 'Turbo-prop' THEN 'não pertence'"
                + "WHEN engine = 'Turbo-shaft' THEN 'não pertence'"
                + "WHEN engine = '4 Cycle' THEN 'não pertence'"
                "ELSE engine END")"""



+-------+----+----+------------+-----+-------+-----+-----+------+
|tailnum|year|type|manufacturer|model|engines|seats|speed|engine|
+-------+----+----+------------+-----+-------+-----+-----+------+
+-------+----+----+------------+-----+-------+-----+-----+------+



# Flights Dataset:

In [37]:
# Importação dos dados para o PySpark:
# Download do http para arquivo local:

!wget --quiet --show-progress https://drive.google.com/file/d/1ggZ_dBMvAkILRh-YrozqsvsXc2XYUYeG/view?usp=sharing

# Carregar dados do Flights:

df_flights = spark.read.csv("./flights.csv", inferSchema=True, header=True)

# Ver algumas informações sobre os tipos de dados de cada coluna:

df_flights.printSchema()


view?usp=sharing.21     [ <=>                ]  67.49K  --.-KB/s    in 0.003s  
root
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- dep_time: string (nullable = true)
 |-- dep_delay: string (nullable = true)
 |-- arr_time: string (nullable = true)
 |-- arr_delay: string (nullable = true)
 |-- carrier: string (nullable = true)
 |-- tailnum: string (nullable = true)
 |-- flight: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- dest: string (nullable = true)
 |-- air_time: string (nullable = true)
 |-- distance: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- minute: string (nullable = true)



In [38]:
df_flights.show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

In [39]:
df_flights.fillna(' ').show()


+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|year|month|day|dep_time|dep_delay|arr_time|arr_delay|carrier|tailnum|flight|origin|dest|air_time|distance|hour|minute|
+----+-----+---+--------+---------+--------+---------+-------+-------+------+------+----+--------+--------+----+------+
|2014|   12|  8|     658|       -7|     935|       -5|     VX| N846VA|  1780|   SEA| LAX|     132|     954|   6|    58|
|2014|    1| 22|    1040|        5|    1505|        5|     AS| N559AS|   851|   SEA| HNL|     360|    2677|  10|    40|
|2014|    3|  9|    1443|       -2|    1652|        2|     VX| N847VA|   755|   SEA| SFO|     111|     679|  14|    43|
|2014|    4|  9|    1705|       45|    1839|       34|     WN| N360SW|   344|   PDX| SJC|      83|     569|  17|     5|
|2014|    3|  9|     754|       -1|    1015|        1|     AS| N612AS|   522|   SEA| BUR|     127|     937|   7|    54|
|2014|    1| 15|    1037|        7|    1

Perguntas<br>
Considere o dataset flights.csv para realizar as seguintes tarefas:

1 - Crie a coluna qa_year_month_day e aponte inconsistências das colunas year , month , day de
acordo com as regras abaixo.<br>
MY : Indica que está com dado faltante no ano.<br>
MM : Indica que está com dado faltante no mes.<br>
MD : Indica que está com dado faltante no dia.<br>
IY : Indica que o valor excede o intervalo [1950,+∞] no ano.<br>
IM : Indica que o valor excede o intervalo [1, 12] no mês.<br>
ID : Indica que o valor excede o intervalo [1, 31] no dia. No mês de Fevereiro o intervalo é [1, 29].

2 - Crie a coluna qa_hour_minute e aponte inconsistencias das colunas hour e minute de acordo
com as regras abaixo.<br>
MH : Indica que está com dado faltante na hora.<br>
MM : Indica que está com dado faltante no minuto.<br>
IH : Indica que o valor excede o intervalo na hora [0, 24].<br>
IM : Indica que o valor excede o intervalo no minuto [0, 59].<br>

3 - Crie a coluna qa_dep_arr_time e aponte inconsistências da coluna dep_time e arr_time de
acordo com as regras abaixo.<br>
MD : Indica que está com dado faltante no dep_time .<br>
MA : Indica que está com dado faltante no arr_time .<br>
FD : Indica que não respeita o formato esperado (HHMM ou HMM) no dep_time .<br>
FA : Indica que não respeita o formato esperado (HHMM ou HMM) no arr_time .<br>

4 - Crie a coluna qa_dep_arr_delay e aponte inconsistências da coluna dep_delay e arr_delay de
acordo com as regras abaixo.<br>
MD : Indica que está com dado faltante no dep_delay .<br>
MA : Indica que está com dado faltante no arr_delay .<br>

5 - Crie a coluna qa_carrier e aponte inconsistências da coluna carrier de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado (2 caracteres alfanuméricos).<br>

6 - Crie a coluna qa_tailnum e aponte inconsistências da coluna tailnum de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
S : Indica que não tem o número de caracteres esperado.<br>
F : Indica que não respeita o formato esperado (ex. N1234Z ou N123AZ).<br>
FN : Indica que não inicia com a letra "N".<br>
FE : Indica que contém caracteres inválidos ("I", "O", ou 0 como primeiro digito).<br>

7 - Crie a coluna qa_flight e aponte inconsistências da coluna flight de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
F : Indica que não respeita o formato esperado (4 caracteres numéricos).<br>

8 - Crie a coluna qa_origin_dest e aponte inconsistências da coluna origin , dest de acordo com
as regras abaixo.<br>
MO : Indica que está com dado faltante no origin .<br>
MD : Indica que está com dado faltante no dest .<br>
FO : Indica que não respeita o formato esperado (3 caracteres alfanuméricos) no origin .<br>
FD : Indica que não respeita o formato esperado (3 caracteres alfanuméricos) no dest .<br>

9 - Crie a coluna qa_air_time e aponte inconsistencias da coluna air_time de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [20, 500].<br>

10 - Crie a coluna qa_distance e aponte inconsistências da coluna distance de acordo com as regras
abaixo.<br>
M : Indica que está com dado faltante.<br>
I : Indica que o valor excede o intervalo [50, 3000].<br>

11 - Crie a coluna qa_distance_airtime e aponte inconsistências entre as colunas distance e
air_time de acordo com as regras abaixo.<br>
M : Indica que está com distance ou air_time faltante.<br>
TL : Indica que a viagem é longa de acordo com a condição:<br>
air_time >= distance × 0.1 + 30.<br>
TS : Indica que a viagem é curta de acordo com a condição:<br>
air_time <= distance × 0.1 + 10.<br>
TR : Indica que a viagem é normal caso as duas anteriores não sejam verdade.<br>